<a href="https://colab.research.google.com/github/salmanhiro/MALT90-Clumps-Classification-Deep-Learning/blob/main/MALT90_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/malt90

/content/drive/MyDrive/malt90


In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True, 
        vertical_flip=True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
!ls

 Aim_ml_split			  malt90fit_classes_Gim
 Albumentation			  malt90fit_Gim_classes
 checkpoint			  malt90image
 checkpoint.data-00000-of-00001   malt90image_Aim
 checkpoint.index		  malt90imagearchive.tar.1
 ckpt.data-00000-of-00001	  malt90image_fitresults
 ckpt.index			  malt90image_Gim
 Gim_ml_split_Gim		  malt90lineinfo.csv
 MALT90_cleaning.ipynb		  malt90lineinfo.csv.1
 malt90fit_Aim_classes		 'MALT90 Training'
 malt90fit_classes


In [ ]:
imgsize = 100

batch_size_train = 64
batch_size_val = 4
batch_size_test = 1
train_generator = train_datagen.flow_from_directory(
        'Gim_ml_split_Gim/train_augmented',
        target_size=(imgsize, imgsize),
        batch_size=batch_size_train,
        class_mode='categorical', 
        color_mode='rgb')

val_generator = val_datagen.flow_from_directory(
        'Gim_ml_split_Gim/val_crop', 
        target_size=(imgsize,imgsize),
        batch_size=batch_size_val,
        class_mode='categorical', 
        color_mode='rgb'
)

test_generator = test_datagen.flow_from_directory(
        'Gim_ml_split_Gim/test_crop',
        target_size=(imgsize,imgsize),
        batch_size=batch_size_test,
        class_mode='categorical',
        color_mode='rgb'
)

Found 3404 images belonging to 5 classes.
Found 275 images belonging to 5 classes.
Found 283 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras import Input, models, layers
from tensorflow.keras.optimizers import Adam

def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(5, activation='softmax')])
    return model
    
model = create_model((imgsize,imgsize,3))

model.compile(Adam(learning_rate=1e-4),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os 


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoint/ckpt.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True, 
    verbose=1)

In [ ]:
model.load_weights("checkpoint/ckpt.h5")

In [ ]:
train_num_samples = train_generator.n
val_num_samples = val_generator.n

history = model.fit(
        train_generator,
        steps_per_epoch= train_num_samples//batch_size_train,
        epochs=500,
        validation_data=val_generator,
        validation_steps= val_num_samples//batch_size_val,
        callbacks=[model_checkpoint_callback])

Epoch 1/500
53/53 [==============================] - ETA: 0s - loss: 0.5161 - accuracy: 0.7940 
Epoch 00001: val_accuracy improved from -inf to 0.55147, saving model to checkpoint/ckpt.h5
53/53 [==============================] - 1289s 25s/step - loss: 0.5161 - accuracy: 0.7940 - val_loss: 1.7592 - val_accuracy: 0.5515
Epoch 2/500
53/53 [==============================] - ETA: 0s - loss: 0.5036 - accuracy: 0.8024
Epoch 00002: val_accuracy improved from 0.55147 to 0.58456, saving model to checkpoint/ckpt.h5
53/53 [==============================] - 63s 1s/step - loss: 0.5036 - accuracy: 0.8024 - val_loss: 1.8942 - val_accuracy: 0.5846
Epoch 3/500
53/53 [==============================] - ETA: 0s - loss: 0.5018 - accuracy: 0.8003
Epoch 00003: val_accuracy did not improve from 0.58456
53/53 [==============================] - 62s 1s/step - loss: 0.5018 - accuracy: 0.8003 - val_loss: 1.8912 - val_accuracy: 0.5809
Epoch 4/500
53/53 [==============================] - ETA: 0s - loss: 0.4973 - accu